In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation,BatchNormalization 
from keras.utils import np_utils
from sklearn.cross_validation import train_test_split 
from sklearn import preprocessing 

from dateutil.parser import parse

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train=pd.read_csv('/root/Downloads/bike/train.csv')
test=pd.read_csv('/root/Downloads/bike/test.csv')

In [3]:
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [4]:
DATETIME = 'datetime'
COUNT = 'count'
CASUAL = 'casual'
REGISTERED = 'registered'

In [5]:
train_y = np.log(train[COUNT].values) 
# train_y = train[COUNT]
train = train.drop(COUNT,axis=1) 

In [6]:
all_data = pd.concat([train, test], axis = 0)
all_data[REGISTERED] = all_data[REGISTERED].fillna(all_data[REGISTERED].mean())
all_data[CASUAL] = all_data[CASUAL].fillna(all_data[CASUAL].mean())
ts = pd.to_datetime(all_data[DATETIME])  
all_data[DATETIME] = ts.dt.hour

In [7]:
X = all_data.values
x_train, x_test, y_train, y_test = train_test_split(X[0:train.shape[0]], train_y, test_size=0.1, random_state=0)

In [8]:
def create_nn_model(input_dim, activation, layers, optimizer):
    model = Sequential()
    l_num = 0
    for l in layers:
        if l_num == 0:
            model.add(Dense(l, input_dim = input_dim, activation = activation, init = 'he_normal'))
        else: 
            model.add(Dense(l, activation = activation, init = 'he_normal')) 
        l_num = l_num + 1 
    model.compile(optimizer = optimizer, loss = 'mse') 
    return model

In [9]:
activations = ['softplus', 'relu', 'linear']
optimizers = ['adagrad', 'rmsprop', 'adam']
layers = [[150, 1], [150, 50, 1], [200, 100, 50, 1]]
nnet_params = []
nnet_score = []

In [10]:
# for a in activations:
#     for o in optimizers:
#         for l in layers:
#             model = create_nn_model(input_dim=X.shape[1], activation=a, layers=l, optimizer=o) 
#             fit = model.fit(x_train, y_train, batch_size=100, nb_epoch=100, validation_split=0.1, verbose=0) 
#             score = np.sqrt(model.evaluate(x_test, y_test)) 
#             print("\nActivation: {} Optimizer: {} Layers: {} Score: {}\n".format(a, o, l, score)) 
#             nnet_params.append(str(a) + '-' + str(o) + '-' + str(l)) 
#             nnet_score.append(score)
#             model.summary()

In [11]:
res = pd.DataFrame({'params': nnet_params, 'score': nnet_score})
res.sort_values(['score'], ascending=True, inplace=True)
print res

Empty DataFrame
Columns: [params, score]
Index: []


In [12]:
model = create_nn_model(input_dim = X.shape[1], activation = 'softplus', layers = [150,  1], optimizer = 'adam') 
fit = model.fit(x_train, y_train, batch_size=24, nb_epoch=100, validation_split=0.1, verbose = 1) 
score = np.sqrt(model.evaluate(x_test, y_test)) 
print  score
model.summary()

Train on 8817 samples, validate on 980 samples
Epoch 1/100
8817/8817 [==============================] - 1s 82us/step - loss: 23.0099 - val_loss: 23.1591
Epoch 2/100
8817/8817 [==============================] - 1s 69us/step - loss: 22.9009 - val_loss: 23.1544
Epoch 3/100
8817/8817 [==============================] - 1s 61us/step - loss: 22.7364 - val_loss: 23.1715
Epoch 4/100
8817/8817 [==============================] - 0s 48us/step - loss: 22.9150 - val_loss: 23.1715
Epoch 5/100
8817/8817 [==============================] - 0s 47us/step - loss: 22.9150 - val_loss: 23.1715
Epoch 6/100
8817/8817 [==============================] - 0s 44us/step - loss: 22.9149 - val_loss: 23.1714
Epoch 7/100
8817/8817 [==============================] - 0s 53us/step - loss: 22.9804 - val_loss: 23.1716
Epoch 8/100
8817/8817 [==============================] - 0s 47us/step - loss: 22.9151 - val_loss: 23.1716
Epoch 9/100
8817/8817 [==============================] - 0s 48us/step - loss: 22.9151 - val_loss: 23.1716

In [13]:
result=model.predict(X[train.shape[0]:])
result=result.flatten()
zz = [pow(np.e,x) for x in result]
result = map(int, zz)
datetime = pd.read_csv('/root/Downloads/bike/sampleSubmission.csv')
df=pd.DataFrame({'datetime': datetime['datetime'],
                   'count': result })
df.to_csv('/root/Downloads/bike/result.csv',index=False)